# Sample Code
## OpenWillis user tutorial

This notebook walks through an interactive demonstration of the basic functions in OpenWillis to process audio and video files. This demo currently addresses use cases with a single speaker. It is intended to help users get a sense of what it's like to work with OpenWillis in a jupyter notebook environment. 

__Note:__ Be sure that you have gone through the OpenWillis [installation steps](https://www.notion.so/brooklynhealth/Installing-OpenWillis-and-jupyter-notebook-14983a8fe047814b88ced7d3831791f2?pvs=12) prior to continuing. 

First, we'll load the necessary libraries. Some warning messages may appear but these can be safely ignored if your environment is set up correctly. 

In [1]:
import openwillis as ow
import tensorflow as tf
import whisperx
import moviepy.editor as mp
import pandas as pd
import os

objc[5247]: Class AVFFrameReceiver is implemented in both /opt/anaconda3/envs/openwillis_test_env/lib/libavdevice.58.8.100.dylib (0x2d0b14798) and /opt/anaconda3/envs/openwillis_test_env/lib/python3.10/site-packages/av/.dylibs/libavdevice.60.1.100.dylib (0x2d3ba8760). One of the two will be used. Which one is undefined.
objc[5247]: Class AVFAudioReceiver is implemented in both /opt/anaconda3/envs/openwillis_test_env/lib/libavdevice.58.8.100.dylib (0x2d0b147e8) and /opt/anaconda3/envs/openwillis_test_env/lib/python3.10/site-packages/av/.dylibs/libavdevice.60.1.100.dylib (0x2d3ba87b0). One of the two will be used. Which one is undefined.


Before getting into the analysis portion, we need to load some data we can work with. For this demonstration, we will use some sample audio and video files of a person reading from a list of [standardized sentences](https://www.cs.columbia.edu/~hgs/audio/harvard.html).

This data can be loaded straight from GitHub into this jupyter notebook environment. 

In [33]:
!git clone https://github.com/bklynhlth/sample_data.git

python(15380) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


fatal: destination path 'sample_data' already exists and is not an empty directory.


Below, we'll organize these files so they are easy to access in the code below. 

In [34]:
audio_dir = 'sample_data/audio_files'
video_dir = 'sample_data/video_files'
baseline_dir = 'sample_data/video_files/baseline_videos'

audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.m4a')]
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
bl_files = [f for f in os.listdir(baseline_dir)]

Let's check to make sure we have the correct files in each of `audio_files`, `video_files`, and `bl_files`. We're only working with data from 5 videos/audio files, so we should expect 3 lists of files that correspond with 'sentences_audio.m4a', 'sentences_video.mp4', and 'sentences_bl_video.mp4', numbered 1-5. 

In [35]:
[audio_files, video_files, bl_files]

[['sentences_4_audio.m4a',
  'sentences_2_audio.m4a',
  'sentences_5_audio.m4a',
  'sentences_3_audio.m4a',
  'sentences_1_audio.m4a'],
 ['sentences_1_video.mp4',
  'sentences_3_video.mp4',
  'sentences_5_video.mp4',
  'sentences_2_video.mp4',
  'sentences_4_video.mp4'],
 ['sentences_2_bl_video.mp4',
  'sentences_5_bl_video.mp4',
  'sentences_4_bl_video.mp4',
  'sentences_3_bl_video.mp4',
  'sentences_1_bl_video.mp4']]

### 1 - Vocal acoustics

#### 1.1 - Single file

Before we continue, you might notice that the audio files in these folders are .m4a. That means we need to first convert to .mp3 or .wav in order to run through OpenWillis. Let's do that first. 

In [5]:
input_folder = 'sample_data/audio_files'
output_folder = 'sample_data/audio_files'

for filename in os.listdir(input_folder):
    if filename.endswith(".m4a"):
      m4a_file = os.path.join(input_folder, filename)
      wav_file = os.path.join(output_folder, filename[:-4] + ".wav") 

      audio = mp.AudioFileClip(m4a_file)
      audio.write_audiofile(wav_file)
      audio.close()

MoviePy - Writing audio in sample_data/audio_files/sentences_4_audio.wav


MoviePy - Done.
MoviePy - Writing audio in sample_data/audio_files/sentences_2_audio.wav


MoviePy - Done.
MoviePy - Writing audio in sample_data/audio_files/sentences_5_audio.wav


MoviePy - Done.
MoviePy - Writing audio in sample_data/audio_files/sentences_3_audio.wav


MoviePy - Done.
MoviePy - Writing audio in sample_data/audio_files/sentences_1_audio.wav


MoviePy - Done.


Now, we can proceed with our processing. First, we'll just process a single audio file from the 'audio_files' folder above. 

In [6]:
framewise, summary = ow.vocal_acoustics(audio_path = 'sample_data/audio_files/sentences_1_audio.wav', voiced_segments = False, option = 'simple')

Now let's take a look at our summary data to make sure it populated correctly. We can first specify that we want to print all rows and columns. 

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
summary

,f0_mean,f0_stddev,f0_range,f1_mean,f1_stddev,f1_range,f2_mean,f2_stddev,f2_range,f3_mean,f3_stddev,f3_range,f4_mean,f4_stddev,f4_range,loudness_mean,loudness_stddev,loudness_range,hnr_mean,hnr_stddev,hnr_range,jitter,jitter_abs,jitter_rap,jitter_ppq5,jitter_ddp,shimmer,shimmer_db,shimmer_apq3,shimmer_apq5,shimmer_apq11,shimmer_dda,gne_ratio,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,mfcc11_mean,mfcc12_mean,mfcc13_mean,mfcc14_mean,mfcc1_var,mfcc2_var,mfcc3_var,mfcc4_var,mfcc5_var,mfcc6_var,mfcc7_var,mfcc8_var,mfcc9_var,mfcc10_var,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,cpp_mean,cpp_var,spir,dur_med,dur_mad,silence_ratio,rel_f0_sd,rel_se0_sd,mean_hrf,std_hrf,mean_naq,std_naq,mean_qoq,std_qoq,fcom,ftrc,fmon,ftrf,ftri,ftrp,ftrcip,ftrps,fcohnr,acom,atrc,amon,atrf,atri,atrp,atrcip,atrps,acohnr
0,72.022328,94.648441,333.12342,855.847492,459.122905,2691.827376,2002.83069,514.591281,3671.664579,3054.021448,514.541926,3474.567592,4113.097143,740.854269,5365.365922,47.273164,28.53317,91.459408,8.902349,6.769389,35.051287,0.029165,0.000163,0.014149,0.01529,0.042446,0.121827,1.189465,0.053654,0.073189,0.111823,0.160961,0.787109,-392.007233,82.052315,-1.6804,21.811811,-10.958357,9.939477,-10.715336,-3.077041,-1.612644,-10.051422,1.564766,-9.352318,1.97057,-3.905676,19413.744141,5899.958984,1315.169556,1374.262451,710.089172,289.615967,328.693817,166.541534,123.671097,180.55484,97.605988,159.249908,60.594902,56.291824,16.037599,5.696161,0.479386,0.941,0.2302,60.056022,0.785983,0.114726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looks good! Notice that some of the final columns are 'NaN' - these features are specifically for the more advanced options and will not populate if 'simple' is specified in the 'option' parameter. 

#### 1.2 - Multiple files

Here, let's go ahead and run vocal acoustics on all 5 files in our folder using a for loop. 

In [9]:
folder_path = 'sample_data/audio_files'

framewise_data = pd.DataFrame()
summary_data = pd.DataFrame()

for filename in os.listdir(folder_path):
  if filename.endswith('.wav'):
    audio_path = os.path.join(folder_path, filename)

    # Run vocal acoustics function
    framewise, summary = ow.vocal_acoustics(audio_path = audio_path, voiced_segments = False, option = 'simple')

    # Here, make sure we can identify each file by adding the name in the first column of the dataframe, remove '.wav' from the name
    filename_no_ext = os.path.splitext(filename)[0]

    # Add filename column as the first column using insert()
    framewise.insert(0, 'filename', filename_no_ext)
    summary.insert(0, 'filename', filename_no_ext)

    # Store results for each file in each dataframe
    framewise_data = pd.concat([framewise_data, framewise], ignore_index=True)
    summary_data = pd.concat([summary_data, summary], ignore_index=True)

We'll examine the data to make sure it's looking okay. 

In [10]:
summary_data.head()

,filename,f0_mean,f0_stddev,f0_range,f1_mean,f1_stddev,f1_range,f2_mean,f2_stddev,f2_range,f3_mean,f3_stddev,f3_range,f4_mean,f4_stddev,f4_range,loudness_mean,loudness_stddev,loudness_range,hnr_mean,hnr_stddev,hnr_range,jitter,jitter_abs,jitter_rap,jitter_ppq5,jitter_ddp,shimmer,shimmer_db,shimmer_apq3,shimmer_apq5,shimmer_apq11,shimmer_dda,gne_ratio,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,mfcc11_mean,mfcc12_mean,mfcc13_mean,mfcc14_mean,mfcc1_var,mfcc2_var,mfcc3_var,mfcc4_var,mfcc5_var,mfcc6_var,mfcc7_var,mfcc8_var,mfcc9_var,mfcc10_var,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,cpp_mean,cpp_var,spir,dur_med,dur_mad,silence_ratio,rel_f0_sd,rel_se0_sd,mean_hrf,std_hrf,mean_naq,std_naq,mean_qoq,std_qoq,fcom,ftrc,fmon,ftrf,ftri,ftrp,ftrcip,ftrps,fcohnr,acom,atrc,amon,atrf,atri,atrp,atrcip,atrps,acohnr
0,sentences_3_audio,91.248045,102.397770,496.774329,812.118256,502.011583,2467.233604,1965.049335,515.977985,3674.364391,2967.765617,546.554891,3707.047856,3855.645922,1077.900561,5414.811572,50.258731,26.461156,87.996063,10.357127,6.782487,39.254406,0.024424,0.000129,0.012155,0.012200,0.036465,0.118077,1.193550,0.048136,0.067372,0.119215,0.144409,0.803483,-408.266357,97.399536,-2.211466,20.501493,-7.419127,15.879626,-9.139483,-2.497345,-1.650639,-10.180691,2.639260,-8.789956,0.788410,-5.317344,19947.832031,6505.985352,1004.004761,1526.311523,859.335205,356.501465,352.268463,132.202026,120.967415,216.006454,117.227242,162.251083,93.303642,73.641899,16.237924,6.170447,0.407456,0.6935,0.1720,52.047556,0.721913,0.169334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sentences_5_audio,83.936418,96.245843,321.054719,832.868747,446.886808,2433.457904,1894.987538,479.486125,2971.421236,2989.500675,457.382907,3175.236215,4039.424853,675.805067,5353.771221,51.318979,26.055988,87.744424,9.178067,6.728781,33.474888,0.028395,0.000154,0.014303,0.015674,0.042909,0.127288,1.180513,0.054518,0.073549,0.119791,0.163553,0.819956,-384.421448,101.546570,-4.359735,17.650267,-11.253160,11.634629,-10.765369,-1.938092,-0.259113,-11.059649,1.936473,-6.532877,1.918104,-5.482338,18191.757812,5939.985352,1190.442993,1250.614502,694.156738,417.522186,263.159607,128.023087,120.608543,167.696960,99.996056,116.087997,72.397781,82.138260,16.142073,6.435658,0.502816,0.7900,0.0930,54.551161,0.706076,0.107596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sentences_2_audio,79.423725,99.962614,492.219863,870.209254,469.441077,2493.844991,1915.006543,492.881692,2807.748257,2978.929868,540.786995,2983.263600,4054.572378,702.206848,5403.690693,49.228088,27.371442,88.175957,8.533875,6.723247,38.821788,0.026612,0.000139,0.012997,0.013301,0.038991,0.116181,1.163537,0.049750,0.069828,0.113077,0.149250,0.799902,-397.645721,92.878609,-12.256537,20.130747,-16.041973,11.664577,-10.372966,-0.614970,-2.620578,-11.105639,4.905626,-9.638748,2.661585,-5.176687,22212.001953,6935.223145,1193.681396,1598.056030,883.040955,327.840424,254.053314,178.640762,183.829559,159.161789,109.541496,143.293793,71.711098,67.496635,16.281701,7.175007,0.481604,0.7810,0.1208,58.524788,0.818678,0.108770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sentences_4_audio,76.785691,92.646901,482.263021,822.884747,465.431924,2848.971861,1988.744307,553.933482,2995.063469,3072.222732,463.417048,3251.663932,4033.462919,838.543279,5417.447200,48.669166,27.047688,86.615024,9.037764,6.474679,32.594939,0.030237,0.000170,0.015005,0.015537,0.045015,0.139183,1.272401,0.059458,0.080026,0.150033,0.178373,0.788116,-409.845459,91.903809,0.008336,26.308781,-8.170308,10.710238,-9.435729,1.415738,3.374910,-9.319065,0.372529,-8.823775,2.426104,-4.272380,19832.345703,6026.186035,1051.059082,1391.214722,836.597351,422.118378,331.008057,111.745148,105.244217,214.087402,97.245270,134.804443,76.726631,60.777683,16

Below, we will save this output as a .csv so we can further analyze and run statistical tests on the output.

In [11]:
output_dir = 'sample_data' # change to your output path
output_filename = 'summary_data.csv'
output_csv_path = os.path.join(output_dir, output_filename)

summary_data.to_csv(output_csv_path, index = False)

### 2 - Speech characteristics

#### 2.1 - Single file

Now we will continue with the speech characteristics function. First, we will need to transcribe our file, here we are using the 'vosk' transcription function:

In [14]:
transcript_json, transcript_text = ow.speech_transcription_vosk(filepath = 'sample_data/audio_files/sentences_1_audio.wav')

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/michelleworthington/.cache/vosk/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /Users/michelleworthington/.cache/vosk/vosk-model-small-en-us-0.15/graph/HCLr.fst /Users/michelleworthington/.cache/vosk/vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo /Users/michelleworthington/.cache/vosk/vos

Then we will pass the JSON file from the transcription function directly to the speech characteristics function:

In [16]:
words, turns, summary_sc = ow.speech_characteristics(json_conf = transcript_json, option = 'simple')

In [17]:
# Examine summary data 

summary_sc

,file_length,speech_length_minutes,speech_length_words,words_per_min,syllables_per_min,mean_pre_word_pause,mean_pause_variability,speech_percentage,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_overall,mattr_5,mattr_10,mattr_25,mattr_50,mattr_100,first_person_percentage,first_person_sentiment_positive,first_person_sentiment_negative,first_person_sentiment_overall,word_repeat_percentage,phrase_repeat_percentage,word_coherence_mean,word_coherence_var,word_coherence_5_mean,word_coherence_5_var,word_coherence_10_mean,word_coherence_10_var,word_coherence_variability_2_mean,word_coherence_variability_2_var,word_coherence_variability_3_mean,word_coherence_variability_3_var,word_coherence_variability_4_mean,word_coherence_variability_4_var,word_coherence_variability_5_mean,word_coherence_variability_5_var,word_coherence_variability_6_mean,word_coherence_variability_6_var,word_coherence_variability_7_mean,word_coherence_variability_7_var,word_coherence_variability_8_mean,word_coherence_variability_8_var,word_coherence_variability_9_mean,word_coherence_variability_9_var,word_coherence_variability_10_mean,word_coherence_variability_10_var,num_turns,num_one_word_turns,mean_turn_length_minutes,mean_turn_length_words,mean_pre_turn_pause,speaker_percentage,first_order_sentence_tangeniality_mean,first_order_sentence_tangeniality_var,second_order_sentence_tangeniality_mean,second_order_sentence_tangeniality_var,turn_to_turn_tangeniality_mean,turn_to_turn_tangeniality_var,turn_to_turn_tangeniality_slope,semantic_perplexity_mean,semantic_perplexity_var,semantic_perplexity_5_mean,semantic_perplexity_5_var,semantic_perplexity_11_mean,semantic_perplexity_11_var,semantic_perplexity_15_mean,semantic_perplexity_15_var,num_interrupts
0,16.53,0.231,42,181.818182,220.779221,0.061463,0.030193,1.397459,0.111,0.0,0.889,0.6124,0.974359,0.888235,0.818947,0.790698,0.790698,0.0,11.1,0.0,11.1,12.121212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.2 - Multiple files

The below code will run the speech characteristics function on multiple files: 

In [18]:
folder_path = 'sample_data/audio_files'

word_data = pd.DataFrame()
turns_data = pd.DataFrame()
summary_sc_data = pd.DataFrame()

for filename in os.listdir(folder_path):
  if filename.endswith('.wav'):
    audio_path = os.path.join(folder_path, filename)

    # Transcribe
    transcript_json, transcript_text = ow.speech_transcription_vosk(filepath = audio_path)
    words, turns, summary_sc = ow.speech_characteristics(json_conf = transcript_json, option = 'simple')

    # Here, make sure we can identify each file by adding the name in the first column of the dataframe, remove '.wav' from the name
    filename_no_ext = os.path.splitext(filename)[0]

    # Add filename column as the first column using insert()
    words.insert(0, 'filename', filename_no_ext)
    turns.insert(0, 'filename', filename_no_ext)
    summary_sc.insert(0, 'filename', filename_no_ext)

    # Store results for each file in each dataframe
    word_data = pd.concat([word_data, words], ignore_index=True)
    turns_data = pd.concat([turns_data, turns], ignore_index=True)
    summary_sc_data = pd.concat([summary_sc_data, summary_sc], ignore_index=True)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/michelleworthington/.cache/vosk/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /Users/michelleworthington/.cache/vosk/vosk-model-small-en-us-0.15/graph/HCLr.fst /Users/michelleworthington/.cache/vosk/vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo /Users/michelleworthington/.cache/vosk/vos

In [19]:
summary_sc_data.head()

,filename,file_length,speech_length_minutes,speech_length_words,words_per_min,syllables_per_min,mean_pre_word_pause,mean_pause_variability,speech_percentage,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_overall,mattr_5,mattr_10,mattr_25,mattr_50,mattr_100,first_person_percentage,first_person_sentiment_positive,first_person_sentiment_negative,first_person_sentiment_overall,word_repeat_percentage,phrase_repeat_percentage,word_coherence_mean,word_coherence_var,word_coherence_5_mean,word_coherence_5_var,word_coherence_10_mean,word_coherence_10_var,word_coherence_variability_2_mean,word_coherence_variability_2_var,word_coherence_variability_3_mean,word_coherence_variability_3_var,word_coherence_variability_4_mean,word_coherence_variability_4_var,word_coherence_variability_5_mean,word_coherence_variability_5_var,word_coherence_variability_6_mean,word_coherence_variability_6_var,word_coherence_variability_7_mean,word_coherence_variability_7_var,word_coherence_variability_8_mean,word_coherence_variability_8_var,word_coherence_variability_9_mean,word_coherence_variability_9_var,word_coherence_variability_10_mean,word_coherence_variability_10_var,num_turns,num_one_word_turns,mean_turn_length_minutes,mean_turn_length_words,mean_pre_turn_pause,speaker_percentage,first_order_sentence_tangeniality_mean,first_order_sentence_tangeniality_var,second_order_sentence_tangeniality_mean,second_order_sentence_tangeniality_var,turn_to_turn_tangeniality_mean,turn_to_turn_tangeniality_var,turn_to_turn_tangeniality_slope,semantic_perplexity_mean,semantic_perplexity_var,semantic_perplexity_5_mean,semantic_perplexity_5_var,semantic_perplexity_11_mean,semantic_perplexity_11_var,semantic_perplexity_15_mean,semantic_perplexity_15_var,num_interrupts
0,sentences_3_audio,14.04,0.2015,41,203.473945,253.101737,0.044805,0.018325,1.435185,0.204,0.0,0.796,0.8176,0.967568,0.909375,0.811765,0.756098,0.756098,0.0,20.4,0.0,20.4,8.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sentences_5_audio,15.18,0.2195,45,205.011390,255.125285,0.046591,0.021181,1.445982,0.216,0.026,0.758,0.9062,0.941463,0.866667,0.758095,0.711111,0.711111,NaN,NaN,NaN,NaN,13.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sentences_2_audio,15.57,0.2260,38,168.141593,234.513274,0.052382,0.023089,1.451509,0.07,0.035,0.896,0.34,0.988235,0.948276,0.888571,0.842105,0.842105,0.0,7.0,0.0,7.0,4.482759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sentences_4_audio,15.75,0.2310,38,164.502165,220.779221,0.080270,0.044303,1.466667,0.114,0.127,0.759,-0.1531,0.96,0.896667,0.837333,0.846154,0.846154,NaN,NaN,NaN,NaN,11.034483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sentences_1_audio,16.53,0.2310,42,181.818182,220.779221,0.061463,0.030193,1.397459,0.111,0.0,0.889,0.6124,0.974359,0.888235,0.818947,0.790698,0.790698,0.0,11.1,0.0,11.1,12.121212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3 - Facial expressivity

#### 3.1 - Single file

From here, let's take a look at the video data. We'll start with just running facial expressivity on a single video file. 

The video used in this example is 18 seconds and estimated runtime of `facial_expressivity` is approximately the same. For longer videos, you should expect runtimes approximately proportionate to the video duration. 

In [57]:
framewise_loc, framewise_disp, summary_fe = ow.facial_expressivity(filepath = 'sample_data/video_files/sentences_1_video.mp4', baseline_filepath = 'sample_data/video_files/baseline_videos/sentences_1_bl_video.mp4')

We can look at the output in a couple of ways. We can look at the `framewise_disp` output to get a sense of displacement for each facial landmark at each frame. This dataframe contains quite a bit of data, so we can also look at the `summary_fe` output which will give us an overall displacement summary for each composite facial area.

In [58]:
framewise_disp.head()

,frame,lmk001,lmk002,lmk003,lmk004,lmk005,lmk006,lmk007,lmk008,lmk009,lmk010,lmk011,lmk012,lmk013,lmk014,lmk015,lmk016,lmk017,lmk018,lmk019,lmk020,lmk021,lmk022,lmk023,lmk024,lmk025,lmk026,lmk027,lmk028,lmk029,lmk030,lmk031,lmk032,lmk033,lmk034,lmk035,lmk036,lmk037,lmk038,lmk039,lmk040,lmk041,lmk042,lmk043,lmk044,lmk045,lmk046,lmk047,lmk048,lmk049,lmk050,lmk051,lmk052,lmk053,lmk054,lmk055,lmk056,lmk057,lmk058,lmk059,lmk060,lmk061,lmk062,lmk063,lmk064,lmk065,lmk066,lmk067,lmk068,lmk069,lmk070,lmk071,lmk072,lmk073,lmk074,lmk075,lmk076,lmk077,lmk078,lmk079,lmk080,lmk081,lmk082,lmk083,lmk084,lmk085,lmk086,lmk087,lmk088,lmk089,lmk090,lmk091,lmk092,lmk093,lmk094,lmk095,lmk096,lmk097,lmk098,lmk099,lmk100,lmk101,lmk102,lmk103,lmk104,lmk105,lmk106,lmk107,lmk108,lmk109,lmk110,lmk111,lmk112,lmk113,lmk114,lmk115,lmk116,lmk117,lmk118,lmk119,lmk120,lmk121,lmk122,lmk123,lmk124,lmk125,lmk126,lmk127,lmk128,lmk129,lmk130,lmk131,lmk132,lmk133,lmk134,lmk135,lmk136,lmk137,lmk138,lmk139,lmk140,lmk141,lmk142,lmk143,lmk144,lmk145,lmk146,lmk147,lmk148,lmk149,lmk150,lmk151,lmk152,lmk153,lmk154,lmk155,lmk156,lmk157,lmk158,lmk159,lmk160,lmk161,lmk162,lmk163,lmk164,lmk165,lmk166,lmk167,lmk168,lmk169,lmk170,lmk171,lmk172,lmk173,lmk174,lmk175,lmk176,lmk177,lmk178,lmk179,lmk180,lmk181,lmk182,lmk183,lmk184,lmk185,lmk186,lmk187,lmk188,lmk189,lmk190,lmk191,lmk192,lmk193,lmk194,lmk195,lmk196,lmk197,lmk198,lmk199,lmk200,lmk201,lmk202,lmk203,lmk204,lmk205,lmk206,lmk207,lmk208,lmk209,lmk210,lmk211,lmk212,lmk213,lmk214,lmk215,lmk216,lmk217,lmk218,lmk219,lmk220,lmk221,lmk222,lmk223,lmk224,lmk225,lmk226,lmk227,lmk228,lmk229,lmk230,lmk231,lmk232,lmk233,lmk234,lmk235,lmk236,lmk237,lmk238,lmk239,lmk240,lmk241,lmk242,lmk243,lmk244,lmk245,lmk246,lmk247,lmk248,lmk249,lmk250,lmk251,lmk252,lmk253,lmk254,lmk255,lmk256,lmk257,lmk258,lmk259,lmk260,lmk261,lmk262,lmk263,lmk264,lmk265,lmk266,lmk267,lmk268,lmk269,lmk270,lmk271,lmk272,lmk273,lmk274,lmk275,lmk276,lmk277,lmk278,lmk279,lmk280,lmk281,lmk282,lmk283,lmk284,lmk285,lmk286,lmk287,lmk288,lmk289,lmk290,lmk291,lmk292,lmk293,lmk294,lmk295,lmk296,lmk297,lmk298,lmk299,lmk300,lmk301,lmk302,lmk303,lmk304,lmk305,lmk306,lmk307,lmk308,lmk309,lmk310,lmk311,lmk312,lmk313,lmk314,lmk315,lmk316,lmk317,lmk318,lmk319,lmk320,lmk321,lmk322,lmk323,lmk324,lmk325,lmk326,lmk327,lmk328,lmk329,lmk330,lmk331,lmk332,lmk333,lmk334,lmk335,lmk336,lmk337,lmk338,lmk339,lmk340,lmk341,lmk342,lmk343,lmk344,lmk345,lmk346,lmk347,lmk348,lmk349,lmk350,lmk351,lmk352,lmk353,lmk354,lmk355,lmk356,lmk357,lmk358,lmk359,lmk360,lmk361,lmk362,lmk363,lmk364,lmk365,lmk366,lmk367,lmk368,lmk369,lmk370,lmk371,lmk372,lmk373,lmk374,lmk375,lmk376,lmk377,lmk378,lmk379,lmk380,lmk381,lmk382,lmk383,lmk384,lmk385,lmk386,lmk387,lmk388,lmk389,lmk390,lmk391,lmk392,lmk393,lmk394,lmk395,lmk396,lmk397,lmk398,lmk399,lmk400,lmk401,lmk402,lmk403,lmk404,lmk405,lmk406,lmk407,lmk408,lmk409,lmk410,lmk411,lmk412,lmk413,lmk414,lmk415,lmk416,lmk417,lmk418,lmk419,lmk420,lmk421,lmk422,lmk423,lmk424,lmk425,lmk426,lmk427,lmk428,lmk429,lmk430,lmk431,lmk432,lmk433,lmk434,lmk435,lmk436,lmk437,lmk438,lmk439,lmk440,lmk441,lmk442,lmk443,lmk444,lmk445,lmk446,lmk447,lmk448,lmk449,lmk450,lmk451,lmk452,lmk453,lmk454,lmk455,lmk456,lmk457,lmk458,lmk459,lmk460,lmk461,lmk462,lmk463,lmk464,lmk465,lmk466,lmk467,lmk468,overall,lower_face,upper_face,lips,eyebrows,mouth_openness
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [59]:
summary_fe

,overall_mean,lower_face_mean,upper_face_mean,lips_mean,eyebrows_mean,mouth_openness_mean,overall_std,lower_face_std,upper_face_std,lips_std,eyebrows_std,mouth_openness_std
0,0.001395,0.001455,0.00136,0.001602,0.001342,0.558199,0.001954,0.002126,0.00195,0.002316,0.001962,0.326606


Just for demonstration, if we don't include a baseline video, the displacement calculations will differ: 

In [60]:
framewise_loc_nobl, framewise_disp_nobl, summary_fe_nobl = ow.facial_expressivity(filepath = 'sample_data/video_files/sentences_1_video.mp4')

summary_fe_nobl

,overall_mean,lower_face_mean,upper_face_mean,lips_mean,eyebrows_mean,mouth_openness_mean,overall_std,lower_face_std,upper_face_std,lips_std,eyebrows_std,mouth_openness_std
0,0.002437,0.002691,0.002294,0.002881,0.002337,0.558199,0.001956,0.002128,0.001951,0.002319,0.001964,0.326606


#### 3.2 - Multiple files

When running this function on multiple video files, make sure to match the video file to the baseline file using a subject identifier. 

In [52]:
folder_path = 'sample_data/video_files'
baseline_folder = 'sample_data/video_files/baseline_videos/'

frames_data = pd.DataFrame()
displacement_data = pd.DataFrame()
summary_fe_data = pd.DataFrame()

for filename in os.listdir(folder_path):
    if filename.endswith('.mp4'):
        video_path = os.path.join(folder_path, filename)
        
        # Extract identifier from filename (assuming a pattern like "person1_video.mp4")
        identifier = "_".join(filename.split("_")[:2])  
        baseline_filename = f"{identifier}_bl_video.mp4"  # Construct baseline filename
        baseline_filepath = os.path.join(baseline_folder, baseline_filename)
        
        # Run facial expressivity - this sample uses the same video as a baseline because the samples are of the same person
        framewise_loc, framewise_disp, summary_fe = ow.facial_expressivity(filepath = video_path, baseline_filepath = baseline_filepath)
    
        # Here, make sure we can identify each file by adding the name in the first column of the dataframe, remove '.mp4' from the name
        filename_no_ext = os.path.splitext(filename)[0]

        # Add filename column as the first column using insert()
        framewise_loc.insert(0, 'filename', filename_no_ext)
        framewise_disp.insert(0, 'filename', filename_no_ext)
        summary_fe.insert(0, 'filename', filename_no_ext)

        # Store results for each file in each dataframe
        frames_data = pd.concat([frames_data, framewise_loc], ignore_index=True)
        displacement_data = pd.concat([displacement_data, framewise_disp], ignore_index=True)
        summary_fe_data = pd.concat([summary_fe_data, summary_fe], ignore_index=True)

In [53]:
summary_fe_data.head()

,filename,overall_mean,lower_face_mean,upper_face_mean,lips_mean,eyebrows_mean,mouth_openness_mean,overall_std,lower_face_std,upper_face_std,lips_std,eyebrows_std,mouth_openness_std
0,sentences_1_video,0.001395,0.001455,0.001360,0.001602,0.001342,0.558199,0.001954,0.002126,0.001950,0.002316,0.001962,0.326606
1,sentences_3_video,0.000445,0.000538,0.000392,0.000653,0.000372,0.499768,0.001087,0.001358,0.001045,0.001592,0.001084,0.299979
2,sentences_5_video,0.000549,0.000639,0.000497,0.000755,0.000503,0.488607,0.001187,0.001332,0.001239,0.001515,0.001354,0.283198
3,sentences_2_video,0.000485,0.000562,0.000441,0.000691,0.000440,0.512052,0.001123,0.001265,0.001169,0.001457,0.001213,0.251655
4,sentences_4_video,0.000699,0.000770,0.000658,0.000897,0.000647,0.492602,0.001652,0.001778,0.001658,0.002016,0.001751,0.309428


### 4 - Emotional expressivity

#### 4.1 - Single file

When running the `emotional_expressivity` function, be aware that the runtime is considerably slower than for `facial_expressivity`. For the 18 second video, processing time is approximately 50 seconds. For longer videos, plan for a processing time of about 2.5x the file length. 

In [61]:
framewise_ee, summary_ee = ow.emotional_expressivity(filepath = 'sample_data/video_files/sentences_1_video.mp4', baseline_filepath = 'sample_data/video_files/baseline_videos/sentences_1_bl_video.mp4')

In [62]:
summary_ee

,angry_mean,disgust_mean,fear_mean,happiness_mean,sadness_mean,surprise_mean,neutral_mean,composite_mean,angry_std,disgust_std,fear_std,happiness_std,sadness_std,surprise_std,neutral_std,composite_std
0,-0.000133,2.260799e-07,0.208517,0.000106,0.093143,0.039584,-0.197796,0.05687,0.001308,0.000003,0.302166,0.000683,0.27325,0.146542,0.21991,0.062916


Again for demonstration without a baseline video, the expressivity metrics will differ: 

In [63]:
framewise_ee_nobl, summary_ee_nobl = ow.emotional_expressivity(filepath = 'sample_data/video_files/sentences_1_video.mp4')

summary_ee_nobl

,angry_mean,disgust_mean,fear_mean,happiness_mean,sadness_mean,surprise_mean,neutral_mean,composite_mean,angry_std,disgust_std,fear_std,happiness_std,sadness_std,surprise_std,neutral_std,composite_std
0,0.000769,2.689349e-07,0.26232,0.000182,0.227167,0.041429,0.468132,0.088645,0.001309,0.000003,0.315619,0.000683,0.306752,0.146802,0.402462,0.067077


#### 4.2 - Multiple files

In [54]:
folder_path = 'sample_data/video_files'
baseline_folder = 'sample_data/video_files/baseline_videos/'  

frames_ee_data = pd.DataFrame()
summary_ee_data = pd.DataFrame()

for filename in os.listdir(folder_path):
    if filename.endswith('.mp4'):
        video_path = os.path.join(folder_path, filename)
        
        # Extract identifier from filename (assuming a pattern like "person1_video.mp4")
        identifier = "_".join(filename.split("_")[:2])  
        baseline_filename = f"{identifier}_bl_video.mp4"  # Construct baseline filename
        baseline_filepath = os.path.join(baseline_folder, baseline_filename)

        # Run emotional expressivity - this sample uses a clip from the same video as a baseline because the samples are of the same person
        framewise_ee, summary_ee = ow.emotional_expressivity(filepath = video_path, baseline_filepath = baseline_filepath)
    
        # Here, make sure we can identify each file by adding the name in the first column of the dataframe, remove '.mp4' from the name
        filename_no_ext = os.path.splitext(filename)[0]

        # Add filename column as the first column using insert()
        framewise_ee.insert(0, 'filename', filename_no_ext)
        summary_ee.insert(0, 'filename', filename_no_ext)

        # Store results for each file in each dataframe
        frames_ee_data = pd.concat([frames_ee_data, framewise_ee], ignore_index=True)
        summary_ee_data = pd.concat([summary_ee_data, summary_ee], ignore_index=True)

In [65]:
summary_ee_data.head()

,filename,angry_mean,disgust_mean,fear_mean,happiness_mean,sadness_mean,surprise_mean,neutral_mean,composite_mean,angry_std,disgust_std,fear_std,happiness_std,sadness_std,surprise_std,neutral_std,composite_std
0,sentences_1_video,-0.000133,2.260799e-07,0.208517,0.000106,0.093143,0.039584,-0.197796,0.056870,0.001308,2.999030e-06,0.302166,0.000683,0.273250,0.146542,0.219910,0.062916
1,sentences_3_video,-0.000562,-1.561678e-08,0.205836,0.000143,0.229009,0.008665,-0.262467,0.073848,0.000619,1.189365e-07,0.272387,0.001624,0.277960,0.066468,0.201298,0.056781
2,sentences_5_video,-0.000538,-3.289343e-08,0.194733,-0.000040,0.169601,0.014741,-0.222928,0.063083,0.000998,6.309327e-08,0.277572,0.000122,0.268884,0.095425,0.210953,0.059760
3,sentences_2_video,-0.000005,1.172024e-07,0.314177,-0.000008,0.171067,0.039969,-0.306117,0.087533,0.003245,2.054223e-06,0.323360,0.000331,0.275496,0.156551,0.191248,0.054921
4,sentences_4_video,-0.000585,9.614122e-09,0.189160,0.000188,0.124286,0.017087,-0.193334,0.055023,0.000504,2.201975e-07,0.280402,0.001255,0.287345,0.095925,0.222452,0.062901
